In [107]:
import pandas as pd
from datetime import datetime

In [108]:
url = 'https://www.anbima.com.br/informacoes/indicadores/'
df = pd.read_html(url, thousands='.', decimal=',')[2]

In [109]:
data_referencia = df.head(1).iloc[0][0]
data_referencia = data_referencia.split('Data e Hora da Última Atualização: ')
data_referencia = data_referencia[1].split(' - ')[0]
data_referencia = datetime.strptime(data_referencia, '%d/%m/%Y')
data_referencia

datetime.datetime(2020, 9, 28, 0, 0)

In [110]:
colunas = [
    'indice',
    'data_referencia',
    'data_captura',
    'descricao',
    'valor'
]
df_base = pd.read_csv(file_path, usecols=colunas)
df_base['data_referencia'] = pd.to_datetime(df_base['data_referencia'])

# se a data da base for maior ou igual a data de captura
if df_base['data_referencia'].max() >= data_referencia:
    print('Arquivo já baixado, saindo')

In [111]:
df['data_captura'] = data_referencia
df.dropna(inplace=True)
# remove o primeiro registro do dataframe
df = df.loc[1:]
df['descricao'] = df[1].copy(deep=False)
df

,0,1,2,data_captura,descricao
2,Estimativa SELIC1,28/09/2020,1.90,2020-09-28,28/09/2020
3,Taxa SELIC do BC2,25/09/2020,1.90,2020-09-28,25/09/2020
4,DI-CETIP3,25/09/2020,1.90,2020-09-28,25/09/2020
6,IGP-M (ago/20)5,Número Índice,832.313,2020-09-28,Número Índice
7,IGP-M (ago/20)5,Var % no mês,2.74,2020-09-28,Var % no mês
8,IGP-M1,Projeção (set/20),4.12,2020-09-28,Projeção (set/20)
9,IPCA (ago/20)6,Número Índice,5357.46,2020-09-28,Número Índice
10,IPCA (ago/20)6,Var % no mês,0.24,2020-09-28,Var % no mês
11,IPCA1,Projeção (set/20),0.49,2020-09-28,Projeção (set/20)
13,Dolar Comercial Compra2,25/09/2020,5.5661,2020-09-28,25/09/2020


In [112]:
df[1] = pd.to_datetime(df[1], format="%d/%m/%Y", errors='coerce')
df

,0,1,2,data_captura,descricao
2,Estimativa SELIC1,2020-09-28,1.90,2020-09-28,28/09/2020
3,Taxa SELIC do BC2,2020-09-25,1.90,2020-09-28,25/09/2020
4,DI-CETIP3,2020-09-25,1.90,2020-09-28,25/09/2020
6,IGP-M (ago/20)5,NaT,832.313,2020-09-28,Número Índice
7,IGP-M (ago/20)5,NaT,2.74,2020-09-28,Var % no mês
8,IGP-M1,NaT,4.12,2020-09-28,Projeção (set/20)
9,IPCA (ago/20)6,NaT,5357.46,2020-09-28,Número Índice
10,IPCA (ago/20)6,NaT,0.24,2020-09-28,Var % no mês
11,IPCA1,NaT,0.49,2020-09-28,Projeção (set/20)
13,Dolar Comercial Compra2,2020-09-25,5.5661,2020-09-28,25/09/2020


In [113]:
df[1].fillna(df['data_captura'], inplace=True)

In [114]:
def formata_indicador(valor, descricao):
    if valor.startswith('Estimativa SELIC'):
        return 'selic_estimativa_anbima'
    if valor.startswith('Taxa SELIC do BC2'):
        return 'selic'
    if valor.startswith('DI-CETIP3'):
        return 'cdi'
    if valor.startswith('IGP-M (') and descricao.startswith('Número Índice'):
        return 'igpm_numero_indice'
    if valor.startswith('IGP-M (') and descricao.startswith('Var % no mês'):
        return 'igpm_variacao_percentual_mes'
    if valor.startswith('IGP-M1') and descricao.startswith('Projeção'):
        return 'igpm_projecao_anbima'
    if valor.startswith('IPCA (') and descricao.startswith('Número Índice'):
        return 'ipca_numero_indice'
    if valor.startswith('IPCA (') and descricao.startswith('Var % no mês'):
        return 'ipca_variacao_percentual_mes'
    if valor.startswith('IPCA1') and descricao.startswith('Projeção'):
        return 'ipca_projecao_anbima'
    if valor.startswith('Dolar Comercial Compra'):
        return 'dolar_comercial_compra'
    if valor.startswith('Dolar Comercial Venda'):
        return 'dolar_comercial_venda'
    if valor.startswith('Euro Compra'):
        return 'euro_compra'
    if valor.startswith('Euro Venda'):
        return 'euro_venda'
    if valor.startswith('TR2'):
        return 'tr'
    if valor.startswith('TBF2'):
        return 'tbf'
    if valor.startswith('FDS4'):
        return 'fds'
    
df[0] = df.apply(lambda x: formata_indicador(x[0], x['descricao']), axis=1)
df

,0,1,2,data_captura,descricao
2,selic_estimativa_anbima,2020-09-28,1.90,2020-09-28,28/09/2020
3,selic,2020-09-25,1.90,2020-09-28,25/09/2020
4,cdi,2020-09-25,1.90,2020-09-28,25/09/2020
6,igpm_numero_indice,2020-09-28,832.313,2020-09-28,Número Índice
7,igpm_variacao_percentual_mes,2020-09-28,2.74,2020-09-28,Var % no mês
8,igpm_projecao_anbima,2020-09-28,4.12,2020-09-28,Projeção (set/20)
9,ipca_numero_indice,2020-09-28,5357.46,2020-09-28,Número Índice
10,ipca_variacao_percentual_mes,2020-09-28,0.24,2020-09-28,Var % no mês
11,ipca_projecao_anbima,2020-09-28,0.49,2020-09-28,Projeção (set/20)
13,dolar_comercial_compra,2020-09-25,5.5661,2020-09-28,25/09/2020


In [115]:
# renomeia e formata as colunas
df.rename(columns={0: 'indice', 1: 'data_referencia', 2: 'valor'}, inplace=True)
df = df[[
    'indice',
    'data_referencia',
    'data_captura',
    'descricao',
    'valor'
]]

# salva o arquivo de saída
df

,indice,data_referencia,data_captura,descricao,valor
2,selic_estimativa_anbima,2020-09-28,2020-09-28,28/09/2020,1.90
3,selic,2020-09-25,2020-09-28,25/09/2020,1.90
4,cdi,2020-09-25,2020-09-28,25/09/2020,1.90
6,igpm_numero_indice,2020-09-28,2020-09-28,Número Índice,832.313
7,igpm_variacao_percentual_mes,2020-09-28,2020-09-28,Var % no mês,2.74
8,igpm_projecao_anbima,2020-09-28,2020-09-28,Projeção (set/20),4.12
9,ipca_numero_indice,2020-09-28,2020-09-28,Número Índice,5357.46
10,ipca_variacao_percentual_mes,2020-09-28,2020-09-28,Var % no mês,0.24
11,ipca_projecao_anbima,2020-09-28,2020-09-28,Projeção (set/20),0.49
13,dolar_comercial_compra,2020-09-25,2020-09-28,25/09/2020,5.5661


In [116]:
# salva o arquivo de saída
file_path = os.path.join('bases', 'indicadores_anbima.csv')
df.to_csv(file_path, mode='a', index=False)